In [1]:
#######################################################################
# This script was written by Nana Owusu, it is meant to preprocess    #
# metabolomic information from CSV files and using the mean absolute  #
# deviation of each treatment group, remove outliers.                 #
#######################################################################
# Modules for text interpretation and math
import os, sys, re, fnmatch
import numpy as np
# Modules for plotting and reading csv
# import matplotlib.pyplot as plt
# from matplotlib.figure import Figure
# import csv
import pandas as pd
# %matplotlib inline
# from ipywidgets import interactive
# Module for GUI
# import tkinter as tk
# from tkinter import filedialog
# Module for saving plots as PDF
# from matplotlib.backends.backend_pdf import PdfPages

## Load csv files with pandas

In [2]:
dataFile = pd.read_csv("/Users/nowusu/uiHackyHour/uihh_nowusu/20190723_chronicrotenoneisradipine_driftnorm.csv")

###     Group metabolite columns by drug treatment

In [3]:
## variable that contains the names of each metabolite measured
# The syntax here is known as a "list comprehension"
columns = [col for col in dataFile.columns if col not in ["Condition", "sample"]]

# gives a Pandas Series of the 21 treatment drugs
groups = dataFile['Condition']

### Get metabolite values sorted into a multi-index format

In [4]:
drug_enumerate = [(groups[0],0)]
count = 0
oldDrug = groups[0]
for i in range(1,len(groups)):
    if oldDrug == groups[i]:
        count += 1
    else :
        count = 0
    drug_enumerate.append((groups[i],count))
    oldDrug = groups[i]


drug_multiIdx = pd.MultiIndex.from_tuples(drug_enumerate)

# the multi-index Pandas object will be added to this dataframe object
# for allow for better group analyses using pandas tools for stats.
metabolites_mi = pd.DataFrame(dataFile[columns])
metabolites_mi.set_axis(labels=drug_multiIdx, axis='index', inplace=True)

In [69]:
metabolites_mi

X1.Octadecanol  X2.Hydroxybutyrate  \
Isradipine          0        1.040873            1.131138   
                    1        1.077633            1.056377   
                    2        0.972901            0.995689   
                    3        1.120797            0.971689   
                    4        1.189482            1.179786   
Isradipine+Rotenone 0        1.051199            1.061541   
                    1        0.951072            1.119360   
                    2        0.985426            1.027503   
                    3        1.023643            1.031288   
Rotenone            0        1.180639            0.953011   
                    1        1.008503            1.095886   
                    2        0.944983            1.128522   
                    3        0.875994            1.005659   
                    4        0.934887            0.955510   
                    5        0.962582            1.226802   
Vehicle             0        0.990781            1.018448   
                    1        0.969146            1.063302   
                    2        0.957327            1.098904   
                    3        1.013684            1.054757   
                    4        0.918264            1.010184   
                    5        0.933773            1.054565   

                       X2.Hydroxyglutarate  X2.Oxoadipate  X3.Hydroxypyruvate  \
Isradipine          0             1.062883       0.959504            1.059183   
                    1             1.463635       1.439655            0.780903   
                    2             1.385253       1.383549            0.362198   
                    3             0.973966       1.136060            1.945651   
                    4             0.983789       1.133995            1.191188   
Isradipine+Rotenone 0             1.543269       1.332296            1.157584   
                    1             1.046026       0.948473            1.242748   
                    2             1.296233       1.326930            0.972275   
                    3             1.156100       1.137521            1.581426   
Rotenone            0             0.437137       0.625078            0.820780   
                    1             0.776436       0.659496            0.760426   
                    2             0.858093       0.821937            0.926124   
                    3             0.956380       0.955579            1.029088   
                    4             1.336476       1.250059            0.838309   
                    5             0.921649       0.803768            0.536962   
Vehicle             0             1.093740       1.003387            1.339430   
                    1             1.273897       1.157104            1.163876   
                    2             1.043531       1.008887            1.639998   
                    3             1.252248       1.285249            1.169168   
                    4             1.050831       1.040321            1.145809   
                    5             0.917510       0.899699            1.134608   

                       X3.Phosphoglycerate  X6.Phosphogluconate  Aconitate  \
Isradipine          0             1.442928             0.937136   1.400339   
                    1             1.327279             1.529427   1.311075   
                    2             1.148379             1.419673   1.231772   
                    3             0.850187             1.211664   1.230800   
                    4             0.475886             1.245228   1.280423   
Isradipine+Rotenone 0             1.108452             1.595933   1.550948   
                    1             0.777225             0.996103   0.923527   
                    2             1.227894             1.419509   1.249656   
                    3             0.645601             1.152196   1.238969   
Rotenone            0             0.652504             0.601315   1.362454   
                    1             1.092

## Functions for calculating group statistics

In [213]:
def getConditions(condGroups):
    ''' Routine for counting how many constituents are in a sequence
    after the first occurrence and saves the constituent as well
    as the count '''
    
    drugs = {}
    for treatment in condGroups:
        if treatment not in drugs:
            drugs[treatment] = 0
        drugs[treatment] += 1
    
    return drugs

def stdErr(group,metabSet):
    # calculate standard deviation for
    # each group    
    metabStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].std(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabStdErr.columns.names = ['Standard Deviation']
    return metabStdErr
    

def meanStdErr(group,metabSet):
    # calculate mean standard error 
    # of each group    
    metabMeanStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].sem(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabMeanStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabMeanStdErr.columns.names = ['Mean Std. Deviation']
    return metabMeanStdErr

def coefOfVar(group,metabStdErr,metabMean):
    # calculate coefficient of variation 
    # of each group    
    metabCoefOfVar = metabStdErr.truediv(other=metabMean,axis='index')
    
#     metabCoefOfVar.set_axis(axis='index', labels=group, inplace=True)
    metabCoefOfVar.columns.names = ['Coeff. of Variation']
    return metabCoefOfVar

def mean(group,metabSet):
    # calculate mean of each group    
    metabMean = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].mean(axis='index')) 
                             for treatment in group], ignore_index=True)
    
    metabMean.set_axis(axis='index', labels=group, inplace=True)
    metabMean.columns.names = ['Mean Std. Deviation']
    return metabMean

def grubbs(group,metabSet,metabMean,metabStdErr):
    
#     grpIdxAble = list(group)
    
#     meanAbsDev = pd.DataFrame(metabSet.loc[grpIdxAble[0],columns].mad(axis='index'))

    meanAbsDev = pd.DataFrame(metabSet,copy=True)
#     operand = metabSet.loc[grpIdxAble[0],columns].sub \
#                 (metabMean.loc[grpIdxAble[0],columns], axis='index')
#     meanAbsDev = pd.DataFrame(operand.div(metabStdErr.loc[grpIdxAble[0],columns], axis='index'))
    
#     for treatment in grpIdxAble[1::]:
#         operand = metabSet.loc[treatment,columns].sub \
#                     (metabMean.loc[treatment,columns], axis='index')
#         meanAbsDev.append(pd.DataFrame(operand.div(metabStdErr.loc[treatment,columns], axis='index')))
    for treatment in group:
            operand = metabSet.loc[treatment,columns].sub \
                        (metabMean.loc[treatment,columns], axis='index')
            meanAbsDev.append(pd.DataFrame(operand.div(metabStdErr.loc[treatment,columns], axis='index')))
        
#         nextGroup = pd.DataFrame(metabSet.loc[treatment,columns].mad(axis='index')).T
#         meanAbsDev.append(nextGroup,ignore_index=True)
    
#     meanAbsDev.set_axis(labels=drug_multiIdx,axis='index',inplace=True)
    
#     meanAbsDev.set_axis(labels=drug_multiIdx, axis='index', inplace=True)
#     lst = []
#     for i in x:
#         upper = i + thresh
#         lower = i - thresh
#         if i > upper or i < lower:
#             lst.append(True)
#         else:
#             lst.append(False)
    
    return meanAbsDev
        
    
# jk_df_groups.get_group('Rotenone')[columns].apply(np.mean)
# jk_df_groups.get_group('Rotenone')[columns].apply(MAD)


In [119]:
tuple(conditions.keys())[0]

'Isradipine'

## Calculate standard deviations

In [59]:
conditions = getConditions(groups)
std = stdErr(conditions,metabolites_mi)
std

Standard Deviation,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
Isradipine,0.081635,0.088200,0.232947,0.198022,0.584402,0.390667,0.226114,0.070003,0.309447,0.665659,...,0.340921,0.297744,0.233368,0.137891,0.090085,0.099384,0.201164,0.372482,0.291293,0.197370
Isradipine+Rotenone,0.043797,0.042452,0.214580,0.182604,0.254977,0.273511,0.268160,0.256213,0.221280,0.517672,...,0.152018,0.237651,0.239118,0.072665,0.304962,0.102556,0.317048,0.324339,0.110495,0.217951
Rotenone,0.105189,0.108737,0.290952,0.228455,0.166712,0.212089,0.247988,0.269809,0.188617,0.531880,...,0.460815,0.158283,0.284253,0.829449,0.081365,0.056950,0.123609,0.354919,0.258844,0.184383
Vehicle,0.035389,0.032227,0.135794,0.135549,0.198383,0.242246,0.187461,0.247503,0.224286,0.386110,...,0.196949,0.283296,0.315509,0.196011,0.216226,0.062737,0.075085,0.194236,0.196260,0.215471


## Calculate Averages

In [54]:
avg = mean(conditions,metabolites_mi)
avg

Mean Std. Deviation,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
Isradipine,1.080337,1.066936,1.173905,1.210552,1.067824,1.048932,1.268625,1.290882,1.186522,1.156080,...,1.214405,1.304215,1.130174,0.811026,1.144503,1.060160,1.244729,1.003088,0.980310,1.130161
Isradipine+Rotenone,1.002835,1.059923,1.260407,1.186305,1.238508,0.939793,1.290935,1.240775,1.203984,0.802797,...,1.109051,1.332972,1.098192,0.830794,1.169790,1.076116,1.436046,0.893629,1.001437,1.143813
Rotenone,0.984598,1.060898,0.881029,0.852653,0.818615,0.963391,0.890017,1.079369,0.732074,1.001246,...,1.057271,1.080459,1.140802,1.327125,0.981871,0.988229,0.800997,1.027838,1.139632,0.907087
Vehicle,0.963829,1.050027,1.105293,1.065775,1.265481,1.188858,1.079686,1.198236,0.916226,0.817197,...,1.140498,1.132253,1.210951,0.839922,0.854348,0.914731,0.825405,0.914690,1.004315,0.924769


## Calculate Coefficient of Variation

In [62]:
cv = coefOfVar(conditions,std,avg)
cv

Coeff. of Variation,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
Isradipine,0.075564,0.082666,0.198437,0.163580,0.547283,0.372442,0.178235,0.054229,0.260802,0.575790,...,0.280731,0.228293,0.206488,0.170020,0.078711,0.093744,0.161612,0.371335,0.297143,0.174639
Isradipine+Rotenone,0.043673,0.040052,0.170247,0.153927,0.205874,0.291033,0.207725,0.206494,0.183790,0.644835,...,0.137070,0.178287,0.217738,0.087465,0.260698,0.095302,0.220778,0.362946,0.110336,0.190547
Rotenone,0.106834,0.102495,0.330241,0.267934,0.203651,0.220148,0.278634,0.249969,0.257648,0.531218,...,0.435853,0.146496,0.249169,0.624997,0.082867,0.057628,0.154318,0.345306,0.227130,0.203269
Vehicle,0.036717,0.030691,0.122858,0.127183,0.156765,0.203763,0.173626,0.206556,0.244793,0.472482,...,0.172687,0.250205,0.260547,0.233368,0.253089,0.068586,0.090967,0.212352,0.195416,0.233000


## Perform Grubb's Analysis

In [214]:
madVals = grubbs(conditions,metabolites_mi,avg,std)
madVals

,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X1.Octadecanol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X2.Hydroxybutyrate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X2.Hydroxyglutarate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X2.Oxoadipate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X3.Hydroxypyruvate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
metabolites_mi.loc['Isradipine',columns] - avg.loc['Isradipine',columns]

,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
0,-0.039465,0.064202,-0.111022,-0.251049,-0.008642,0.393996,-0.331490,0.109457,-0.320746,-0.696788,...,-0.367585,-0.076283,0.106119,0.020938,-0.148096,-0.117643,0.138299,-0.456421,-0.071980,-0.276903
1,-0.002704,-0.010559,0.289729,0.229102,-0.286922,0.278347,0.260801,0.020193,-0.068721,-0.453440,...,0.051170,0.494195,0.053444,-0.098106,0.088706,-0.095402,0.130440,-0.109523,-0.234250,0.167580
2,-0.107436,-0.071247,0.211348,0.172996,-0.705626,0.099447,0.151048,-0.059110,-0.186372,0.017993,...,-0.037297,0.023372,-0.373244,-0.078435,-0.011586,0.043450,0.040259,0.333120,-0.218724,0.212108
3,0.040460,-0.095247,-0.199939,-0.074493,0.877826,-0.198745,-0.056961,-0.060082,0.481439,1.035151,...,-0.185929,-0.278288,0.251035,0.232291,0.037397,0.096998,-0.349903,0.429927,0.478952,-0.071267
4,0.109145,0.112851,-0.190116,-0.076558,0.123363,-0.573046,-0.023398,-0.010458,0.094400,0.097085,...,0.539641,-0.162995,-0.037354,-0.076687,0.033579,0.072598,0.040905,-0.197103,0.046002,-0.031518


In [188]:
trial = metabolites_mi.loc['Isradipine',columns].sub(avg.loc['Isradipine',columns])
trial_mad = trial.div(std.loc['Isradipine',columns])

In [189]:
trial_mad

,X1.Octadecanol,X2.Hydroxybutyrate,X2.Hydroxyglutarate,X2.Oxoadipate,X3.Hydroxypyruvate,X3.Phosphoglycerate,X6.Phosphogluconate,Aconitate,Adenine,Adenosine,...,Taurine,Threonine,Thymine,Tryptophan,Tyrosine,Uracil,Urea,Uridine,Valine,Xanthine
0,-0.483430,0.727916,-0.476599,-1.267780,-0.014787,1.008523,-1.466029,1.563599,-1.036512,-1.046765,...,-1.078210,-0.256202,0.454729,0.151846,-1.643953,-1.183729,0.687494,-1.225349,-0.247106,-1.402960
1,-0.033128,-0.119714,1.243757,1.156953,-0.490966,0.712493,1.153406,0.288463,-0.222078,-0.681190,...,0.150093,1.659798,0.229011,-0.711476,0.984684,-0.959939,0.648429,-0.294037,-0.804173,0.849061
2,-1.316058,-0.807791,0.907281,0.873620,-1.207432,0.254557,0.668017,-0.844387,-0.602274,0.027030,...,-0.109399,0.078497,-1.599379,-0.568824,-0.128611,0.437191,0.200130,0.894325,-0.750874,1.074670
3,0.495626,-1.079901,-0.858303,-0.376183,1.502092,-0.508732,-0.251915,-0.858276,1.555805,1.555076,...,-0.545372,-0.934657,1.075705,1.684602,0.415129,0.975993,-1.739394,1.154223,1.644230,-0.361084
4,1.336991,1.279490,-0.816137,-0.386611,0.211093,-1.466842,-0.103478,-0.149400,0.305059,0.145849,...,1.582889,-0.547435,-0.160067,-0.556147,0.372752,0.730483,0.203343,-0.529162,0.157923,-0.159688
